# Scraping Job Postings from LinkedIn

In [19]:
#Import packages
import time, os
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup as bs
import requests
import re
import pickle

#Hide Warnings
import warnings
warnings.filterwarnings('ignore')

In [20]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
source = 'https://www.linkedin.com/jobs'
driver.get(source)

# Get Location to Rancho Cordova
location_box_clear = driver.find_element_by_xpath('//*[@id="JOBS"]/section[2]/button')
location_box_clear.click()
location_box = driver.find_element_by_xpath('//*[@id="JOBS"]/section[2]/input')
location_box.click()
location_box.send_keys("Rancho Cordova, California, United States")
location_box.send_keys(Keys.RETURN)

In [21]:
# past two weeks 
time_dropdown = driver.find_element_by_xpath('//*[@id="jserp-filters"]/ul/li[1]/div/div/button')
time_dropdown.click()
past_month_button = driver.find_element_by_xpath('//*[@id="jserp-filters"]/ul/li[1]/div/div/div/fieldset/div/div[3]/label')
past_month_button.click()
time_done_button = driver.find_element_by_xpath('//*[@id="jserp-filters"]/ul/li[1]/div/div/div/button')
time_done_button.click()

# pause in action or linkedin will jump to sign in page
time.sleep(5)

# within 10 miles
distance_dropdown = driver.find_element_by_xpath('//*[@id="jserp-filters"]/ul/li[2]/div/div/button')
distance_dropdown.click()
filter_10mi = driver.find_element_by_xpath('//*[@id="jserp-filters"]/ul/li[2]/div/div/div/fieldset/div/div[1]/label')
filter_10mi.click()
distance_done_button = driver.find_element_by_xpath('//*[@id="jserp-filters"]/ul/li[2]/div/div/div/button')
distance_done_button.click()

In [17]:
## Can specify these later, but for now we'll scrape the first 1000 jobs within 25 miles of RC

# Company
# Salary
# Location
# Job Type
# Experience Level
# On-site/Remote

In [22]:
#How many jobs are curently available within 10 miles of Rancho Cordova on LinkedIn
no_of_jobs = driver.find_element_by_css_selector('h1>span').get_attribute('innerText')

print('There are', no_of_jobs, 'jobs available within 10 miles of Rancho Cordova on LinkedIn over the past month.')

There are 13,000+ jobs available within 10 miles of Rancho Cordova on LinkedIn over the past month.


In [27]:
#Browse all jobs for the search.
# Set pause time
SCROLL_PAUSE_TIME = 10
last_height = driver.execute_script("return document.body.scrollHeight")
while True: 
    #Scroll until hit the see more jobs button.
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    try:
        #Click the see more jobs button and then keep scrolling.
        driver.find_element_by_xpath('//*[@id="main-content"]/section/button').click()
        time.sleep(15)
        print("clicked loading button")
    except:
        pass
        time.sleep(15)
        print("no loading button")
        
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Stop the scrolling and button clicking if the page isn't loading more jobs
    if new_height == last_height:
        break
    last_height = new_height

clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button
clicked loading button


In [ ]:
##### Create a list of all jobs in the search.

In [28]:
#Create a list of the jobs.
job_lists = driver.find_element_by_class_name('jobs-search__results-list')
jobs = job_lists.find_elements_by_tag_name('li')

In [29]:
#Test that it collected all jobs.
#If it significantly dropped, the sleep.time time may need to be increased to allow:
#More loading time or
#Not set off restrictions for the site.
print(len(jobs), 'were collected from the search')

998 were collected from the search


In [31]:
#Pull basic information from each job.
job_title = []
company_name = []
date = []
job_link = []

for job in jobs:
    job_title0 = job.find_element_by_css_selector('h3').get_attribute('innerText')
    job_title.append(job_title0)
 
    company_name0 = job.find_element_by_css_selector('h4').get_attribute('innerText')
    company_name.append(company_name0)
 
    date0 = job.find_element_by_css_selector('div>div>time').get_attribute('datetime')
    date.append(date0)
    
    job_link0 = job.find_element_by_css_selector('a').get_attribute('href')
    job_link.append(job_link0)

In [32]:
#See first 5 of each for verification.
print('Job Titles:',job_title[:5])
print(' ')
print('Company Names:',company_name[:5])
print(' ')
print('Date:', date[:5])

Job Titles: ['Patient Services Representative', 'Medical Office Representative', 'Bookseller', 'Planning Analyst', 'Underwriting Analyst Trainee - Commercial Lines - Work from Home']
 
Company Names: ['Sutter Health', 'Dignity Health', 'Half Price Books', 'Intel Corporation', 'Berkshire Hathaway GUARD Insurance Companies']
 
Date: ['2022-02-05', '2022-02-05', '2022-01-10', '2022-02-05', '2022-02-02']


In [ ]:
# #### maybe use bs on the job_link instead of clicking here
# #Inital job description and criteria lists.
# jd = []
# cl = []
# #Get job criteria.

# #Do in batches that end with longer timers 
# #to work around StaleElementReferenceException.
# batchsize = 10

# for i in range(0, len(jobs), batchsize):
    
#     batch = jobs[i:i+batchsize]

#     for job in batch:
       
#         job.click()
        
#         detail_path = 'description__job-criteria-list'
#         details = driver.find_element_by_class_name(detail_path).get_attribute('innerText')
#         cl.append(details)
    
#         time.sleep(5)
        
#     time.sleep(30)

In [ ]:
#Verify criteria is correct.
# print(cl[0])

In [ ]:
#Get job descriptions.

# #Do in batches that end with longer timers 
# #to work around StaleElementReferenceException.
# batchsize = 10

# for i in range(0, len(jobs), batchsize):
    
#     batch = jobs[i:i+batchsize]

#     for job in batch:
        
#         job.click()
    
#         jd_path = 'show-more-less-html__markup'
#         jd0 = driver.find_element_by_class_name(jd_path).get_attribute('innerText')
#         jd.append(jd0)
    
#         time.sleep(1)
        
#     time.sleep(5)

In [ ]:
#Verify job description is correct.
# print(jd[0])

In [33]:
# Verify that all lists are the same length.
print(len(job_title))
print(len(company_name))
print(len(date))
print(len(job_link))
# print(len(cl))
# print(len(jd))

998
998
998
998


In [34]:
# Create and save a dataframe of the collected data.
job_post_data = pd.DataFrame({'Date': date,
                              'Company': company_name,
                              'Title': job_title,
                              'Job Link': job_link})



In [35]:
job_post_data.head()

,Date,Company,Title,Job Link
0,2022-02-05,Sutter Health,Patient Services Representative,https://www.linkedin.com/jobs/view/patient-ser...
1,2022-02-05,Dignity Health,Medical Office Representative,https://www.linkedin.com/jobs/view/medical-off...
2,2022-01-10,Half Price Books,Bookseller,https://www.linkedin.com/jobs/view/bookseller-...
3,2022-02-05,Intel Corporation,Planning Analyst,https://www.linkedin.com/jobs/view/planning-an...
4,2022-02-02,Berkshire Hathaway GUARD Insurance Companies,Underwriting Analyst Trainee - Commercial Line...,https://www.linkedin.com/jobs/view/underwritin...


In [36]:
len(job_post_data)

998

In [37]:
job_post_data.to_csv('LinkedIn_Job_Postings.csv', index = False)

In [ ]:
# read HTML content with bs4
soup = bs(driver.page_source, 'html.parser')
# contents_div = soup.find('div', class_='columns is-multiline is-mobile js--reviews--target')
# # check amount of movie reviews
# len(contents_div.find_all('div', class_='review-stack'))